In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import random
import cv2
import os
import argparse
from imutils import paths
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [53]:
from glob import glob

In [21]:
%run lenet.ipynb
{
 "cells": [
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "collapsed": False
   },
   "outputs": [],
   "source": [
    "from keras.models import Sequential\n",
    "from keras.layers.convolutional import Conv2D, MaxPooling2D\n",
    "from keras.layers.core import Activation, Flatten, Dense\n",
    "from keras import backend as K"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {
    "collapsed": False
   },
   "outputs": [],
   "source": [
    "class LeNet:\n",
    "    \n",
    "    def build(width, height, depth, classes):\n",
    "        # initialize the model\n",
    "        model = Sequential()\n",
    "        inputShape = (height, width, depth)\n",
    "        \n",
    "        # if using 'channels first' update input shape\n",
    "        if K.image_data_format() == \"channels_first\":\n",
    "            inputShape = (depth, height, width)\n",
    "            \n",
    "        # first set of CONV -> ReLU -> POOL\n",
    "        model.add(Conv2D(20, (5,5), padding='same', input_shape=inputShape))\n",
    "        model.add(Activation(\"relu\"))\n",
    "        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))\n",
    "        \n",
    "        # second set\n",
    "        model.add(Conv2D(50, (5,5), padding='same'))\n",
    "        model.add(Activation(\"relu\"))\n",
    "        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))\n",
    "        \n",
    "        # FC -> ReLU\n",
    "        model.add(Flatten())\n",
    "        model.add(Dense(500))\n",
    "        model.add(Activation(\"relu\"))\n",
    "        \n",
    "        # Softmax classifier\n",
    "        model.add(Dense(classes))\n",
    "        model.add(Activation(\"softmax\"))\n",
    "        \n",
    "        # return network architecture\n",
    "        return model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "collapsed": True
   },
   "outputs": [],
   "source": [
    "\n"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.6.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}


{'cells': [{'cell_type': 'code',
   'execution_count': 3,
   'metadata': {'collapsed': False},
   'outputs': [],
   'source': ['from keras.models import Sequential\n',
    'from keras.layers.convolutional import Conv2D, MaxPooling2D\n',
    'from keras.layers.core import Activation, Flatten, Dense\n',
    'from keras import backend as K']},
  {'cell_type': 'code',
   'execution_count': 6,
   'metadata': {'collapsed': False},
   'outputs': [],
   'source': ['class LeNet:\n',
    '    \n',
    '    def build(width, height, depth, classes):\n',
    '        # initialize the model\n',
    '        model = Sequential()\n',
    '        inputShape = (height, width, depth)\n',
    '        \n',
    "        # if using 'channels first' update input shape\n",
    '        if K.image_data_format() == "channels_first":\n',
    '            inputShape = (depth, height, width)\n',
    '            \n',
    '        # first set of CONV -> ReLU -> POOL\n',
    "        model.add(Conv2D(20, (5,5), pad

In [40]:
# number of epochs, learning rate, batch size
EPOCHS = 25
INIT_LR = 1e-3
BS = 32

In [189]:
# initialize the data and labels
data = []
labels = []

In [159]:
basepath = os.path.join('..', 'train\\')

In [160]:
basepath

'..\\train\\'

In [182]:
%%time
imagePaths = []
for path in sorted(glob(basepath + "\*")):
    # escape \
    cervix_type = path.split("\\")[-1]
    #print(cervix_type)
    imagefolders = (glob(basepath + path + "\\*"))
    imagePaths = imagePaths + imagefolders
 

#random.seed(42)
#random.shuffle(imagePaths)

print(imagePaths)


['..\\train\\..\\train\\Type_1\\0.jpg', '..\\train\\..\\train\\Type_1\\10.jpg', '..\\train\\..\\train\\Type_1\\1013.jpg', '..\\train\\..\\train\\Type_1\\1014.jpg', '..\\train\\..\\train\\Type_1\\1019.jpg', '..\\train\\..\\train\\Type_1\\102.jpg', '..\\train\\..\\train\\Type_1\\1023.jpg', '..\\train\\..\\train\\Type_1\\1024.jpg', '..\\train\\..\\train\\Type_1\\1026.jpg', '..\\train\\..\\train\\Type_1\\1027.jpg', '..\\train\\..\\train\\Type_1\\1033.jpg', '..\\train\\..\\train\\Type_1\\104.jpg', '..\\train\\..\\train\\Type_1\\1040.jpg', '..\\train\\..\\train\\Type_1\\1056.jpg', '..\\train\\..\\train\\Type_1\\1059.jpg', '..\\train\\..\\train\\Type_1\\1061.jpg', '..\\train\\..\\train\\Type_1\\1070.jpg', '..\\train\\..\\train\\Type_1\\1071.jpg', '..\\train\\..\\train\\Type_1\\1077.jpg', '..\\train\\..\\train\\Type_1\\1079.jpg', '..\\train\\..\\train\\Type_1\\109.jpg', '..\\train\\..\\train\\Type_1\\1093.jpg', '..\\train\\..\\train\\Type_1\\1100.jpg', '..\\train\\..\\train\\Type_1\\1105.jpg',

In [190]:
len(imagePaths)

1480

In [191]:
%%time
for imagepath in imagePaths:
    image = cv2.imread(imagepath)
    image = cv2.resize(image, (28, 28))
    image = img_to_array(image)
    data.append(image)
    
    label = imagepath.split(os.path.sep)[-2]
    if label == 'Type_1':
        label = 1
    elif label == 'Type_2':
        label = 2
    else:
        label = 3
    
    labels.append(label)

KeyboardInterrupt: 

In [93]:
from collections import Counter

In [100]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [192]:
# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data,
labels, test_size=0.25, random_state=42)

In [194]:
# convert the labels from integers to vectors
trainY = to_categorical(trainY, num_classes=3)
testY = to_categorical(testY, num_classes=3)

IndexError: index 3 is out of bounds for axis 1 with size 3

In [195]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
horizontal_flip=True, fill_mode="nearest")

In [196]:
# initialize the model
print("[INFO] compiling model...")
model = LeNet.build(width=28, height=28, depth=3, classes=2)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,
metrics=["accuracy"])

[INFO] compiling model...


In [197]:
# train the network
print("[INFO] training network...")
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
epochs=EPOCHS, verbose=1)

[INFO] training network...


AttributeError: 'list' object has no attribute 'shape'